# Project 2

## Predicting NFL All-Pros

In [ ]:
import pandas as pd

Here we are collecting the information that www.pro-football-reference.com has on All-Pros from 1970 (year of the merger)
until 2021 (last completed season).

In [ ]:
df_ap = pd.DataFrame()

for year in range(1970, 2022):
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/allpro.htm'

    df_url = pd.read_html(url)[0]

    df_url.drop(columns = ['Yrs', 'Cmp', 'Att', 'Yds', 'TD', 'Int', 'Att.1', 'Yds.1', 'TD.1', 
                           'Rec', 'Yds.2', 'TD.2', 'Solo', 'Sk', 'Int.1'], inplace = True)
    
    df_url['Year'] = year

    df_temp = [df_ap, df_url]

    df_ap = pd.concat(df_temp)
    


In [ ]:
df_ap.rename(columns = {'All-pro teams': 'all_pro'}, inplace = True)

Dropping everything but QB since we are focusing on quaterbacks.

In [ ]:
df_ap = df_ap[df_ap['Pos'] == 'QB']

Dropping all of the rows where a player did not make a AP (Associated Press) team.

In [ ]:
df_ap_trimmed = df_ap.loc[df_ap['all_pro'].str.contains('AP')]

Creating one hot columns for AP 1st and 2nd Teams.

In [ ]:
df_ap_trimmed['ap_1st'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if (' AP: 1st Tm' in teams.split(',') or 'AP: 1st Tm' in teams.split(',')) else 0)
df_ap_trimmed['ap_2nd'] = df_ap_trimmed.all_pro.apply(lambda teams : 1 if (' AP: 2nd Tm' in teams.split(',') or 'AP: 2nd Tm' in teams.split(',')) else 0)

Dropping everyone who wasn't on AP 1st or 2nd team.

In [ ]:
df_ap = df_ap_trimmed[df_ap_trimmed['ap_1st'] + df_ap_trimmed['ap_2nd'] > 0]

Dropping the 'all_pro' column because we don't need it anymore.

In [ ]:
df_ap.drop(columns = ['all_pro'], inplace = True)

In [ ]:
df_ap.columns = map(str.lower, df_ap.columns)

Removing Dave Lewis, because he was AP 1st Team as a punter in 1970, but was listed as a QB.

In [ ]:
df_ap = df_ap.loc[df_ap['player'] != 'Dave Lewis']

Pulling the passing data from www.pro-football-reference.com

In [ ]:
df_pass = pd.DataFrame()

for year in range(1970, 2022):
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/passing.htm'

    df_url = pd.read_html(url)[0]
    
    df_url = df_url[df_url['Pos'].str.upper() == 'QB']

    df_url['year'] = year

    df_temp = [df_pass, df_url]

    df_pass = pd.concat(df_temp)

Removing the QBR and 1D columns, because they don't exist for every year.

In [ ]:
df_pass.drop(columns = ['QBR', '1D'], inplace = True)

In [ ]:
df_pass.columns = map(str.lower, df_pass.columns)

Splitting 'qbrec' into wins, loses, and ties. And then dropping 'qbrec'.

In [ ]:
df_pass[['wins', 'loses', 'ties']] = df_pass['qbrec'].str.split('-', expand = True)

In [ ]:
df_pass.drop(columns = 'qbrec', inplace = True)

Capitalizing all of 'pos' so it is consistent.

In [ ]:
df_pass['pos'] = df_pass['pos'].apply(str.upper)

The website we're pulling from using '*' and '+' appended to names to represent different awards, so 
we need to remove them so we can join later.

In [ ]:
df_pass['player'] = df_pass['player'].apply(lambda x : x.replace('*', ''))

In [ ]:
df_pass['player'] = df_pass['player'].apply(lambda x : x.replace('+', ''))

Replacing NaN's with 0, because if the value is missing, it means they did not have any of that category.

In [ ]:
df_pass.fillna(0, inplace = True)

Renaming columns, mostly to differentiate them from the rushing stats later.

In [ ]:
df_pass.rename(columns = {'att': 'pass_att', 'yds': 'pass_yds', 'td': 'pass_td', 'lng': 'pass_lng', 'y/a': 'pass_y/a',
                           'y/g': 'pass_y/g', 'yds.1': 'yards_lost_sack', 'rk': 'pass_rk'}, inplace = True)

In [ ]:
df_rush = pd.DataFrame()

for year in range(1970, 2022):
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/rushing.htm'

    df_url = pd.read_html(url)[0]

    df_url = df_url[df_url[( 'Unnamed: 4_level_0',    'Pos')].str.upper() == 'QB']

    df_url['Year'] = year

    df_temp = [df_rush, df_url]

    df_rush = pd.concat(df_temp)

The table was formatted weird, so I'm just manually renaming all the columns.

In [ ]:
df_rush.columns = ['rush_rk', 'player', 'tm', 'age', 'pos', 'g', 'gs', 'rush_att', 'rush_yds', 
                   'rush_td', 'rush_lng', 'rush_y/a', 'rush_y/g', 'fmb.1', 'year', 'drop', 'fmb.2']

Drop 1D (first downs), because it was not always tracked.

In [ ]:
df_rush.drop(columns = ['drop'], inplace = True)

In [ ]:
df_pass['player'] = df_pass['player'].apply(lambda x : x.replace('*', ''))
df_pass['player'] = df_pass['player'].apply(lambda x : x.replace('+', ''))

All other stats should be 0 if they don't exist.

In [ ]:
df_rush.fillna(0, inplace = True)

Because the table was formatted weirdly, it store fumbles into two different columns. It seems to have only
stored it in one column each year, so I was able to just add them together to fix it.

In [ ]:
df_rush['fmb'] = df_rush['fmb.1'].astype('int') + df_rush['fmb.2'].astype('int')

In [ ]:
df_rush.drop(columns = ['fmb.1', 'fmb.2'], inplace = True)

In [ ]:
df_rush['pos'] = df_rush['pos'].apply(str.upper)

Using outer joins to combine the tables.

In [ ]:
df_p_r = pd.merge(df_pass, df_rush, how = 'left', on = ['player', 'year', 'tm', 'pos', 'age', 'pos', 'g', 'gs'])

In [ ]:
df_p_r[['age', 'g', 'gs']] = df_p_r[['age', 'g', 'gs']].astype(int)

In [ ]:
df = pd.merge(df_p_r, df_ap, how = "left", on = ['player', 'year', 'tm', 'pos', 'age', 'g', 'gs'] )

In [ ]:
df.fillna(0, inplace = True)

In [ ]:
df[['ap_1st', 'ap_2nd', 'fmb']] = df[['ap_1st', 'ap_2nd', 'fmb']].astype(int)

In [ ]:
df = df[['year', 'player', 'tm', 'age', 'pos', 'g', 'gs', 'wins', 'loses', 'ties', 'pass_rk', 'cmp', 'pass_att', 'cmp%', 'pass_yds',
         'pass_td', 'td%', 'int', 'int%', 'pass_lng', 'pass_y/a', 'ay/a', 'y/c', 'pass_y/g', 'rate',
         'sk', 'yards_lost_sack', 'sk%', 'ny/a', 'any/a', '4qc', 'gwd', 'rush_rk', 'rush_att', 'rush_yds',
         'rush_td', 'rush_lng', 'rush_y/a', 'rush_y/g', 'fmb', 'ap_1st', 'ap_2nd' ]]

In [ ]:
df.to_csv('nfl_qb.csv', index = False)